In [107]:
import numpy as np
import os
import glob
import SimpleITK as sitk
from skimage import measure
import nibabel as nib
import itk
from itkwidgets import view
import seaborn as sns
from mitok.utils.mdicom import SERIES

def connected_domain(image, mask=True):
    cca = sitk.ConnectedComponentImageFilter()
    cca.SetFullyConnected(True)
    _input = sitk.GetImageFromArray(image.astype(np.uint8))
    output_ex = cca.Execute(_input)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(output_ex)
    num_label = cca.GetObjectCount()
    num_list = [i for i in range(1, num_label+1)]
    area_list = []
    for l in range(1, num_label +1):
        area_list.append(stats.GetNumberOfPixels(l))
    num_list_sorted = sorted(num_list, key=lambda x: area_list[x-1])[::-1]
    largest_area = area_list[num_list_sorted[0] - 1]
    final_label_list = [num_list_sorted[0]]

    for idx, i in enumerate(num_list_sorted[1:]):
        if area_list[i-1] >= (largest_area//10):
            final_label_list.append(i)
        else:
            break
    output = sitk.GetArrayFromImage(output_ex)

    for one_label in num_list:
        if  one_label in final_label_list:
            continue
        x, y, z, w, h, d = stats.GetBoundingBox(one_label)
        one_mask = (output[z: z + d, y: y + h, x: x + w] != one_label)
        output[z: z + d, y: y + h, x: x + w] *= one_mask

    if mask:
        output = (output > 0).astype(np.uint8)
    else:
        output = ((output > 0)*255.).astype(np.uint8)
    return output

def connected_component(image):
    # 标记输入的3D图像
    label, num = measure.label(image, connectivity=1, return_num=True)
    if num < 1:
        return [], image
        
    # 获取对应的region对象
    region = measure.regionprops(label)
    # 获取每一块区域面积并排序
    num_list = [i for i in range(1, num+1)]
    area_list = [region[i-1].area for i in num_list]
    num_list_sorted = sorted(num_list, key=lambda x: area_list[x-1])[::-1]
    # 去除面积较小的连通域
    '''
    if len(num_list_sorted) > 3:
        # for i in range(3, len(num_list_sorted)):
        for i in num_list_sorted[3:]:
            # label[label==i] = 0
            label[region[i-1].slice][region[i-1].image] = 0
        num_list_sorted = num_list_sorted[:3]
    '''
    return num_list_sorted, label
   
def np_count(nparray, x):
    i = 0
    for n in nparray:
        if n == x:
            i += 1
    return i

def remove_mix_region(num, volume, out_data):
    after_num = []
    after_volume = []
    for i in range(len(num)):
        if volume[i] > 0:
            after_num.append(num[i])
            after_volume.append(volume[i])
        else:
            out_data[out_data==num[i]] = 0
    return after_num, after_volume, out_data

def save_nii(plaque, data):
    dealt_plaque_dir = plaque.split('mask_source/')[0]
    data[np.nonzero(data)] = 1
    plaque_nii = nib.Nifti1Image(data, np.eye(4))
    nib.save(plaque_nii, os.path.join(dealt_plaque_dir, 'mask_source/mask_plaque_dealt.nii.gz'))            
    
def get_brightness(after_num, after_data, img_tensor):
    brightness = []
    for i in after_num:
        brightness.append(round(img_tensor[after_data==i].mean()))
        #print(img_tensor[after_data==i])
    return brightness

plaque_dir = '/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata' 
plaque_list = glob.glob(os.path.join(plaque_dir, '*', '*', '*'+'_CTA', 'mask_source/mask_plaque.nii.gz'))
counts = 0
without_plaque_list = []
all_plaque_volumes = []
all_plaque_brightness = []
for plaque in plaque_list:
    #print(plaque.split('/')[6])
    if int(plaque.split('/')[6]) < 1020000:
        continue
    if counts == 1:
        break
    print(plaque)
    data = sitk.ReadImage(plaque)
    data = sitk.GetArrayFromImage(data)
    #print(data)
    #num, out_data = connected_domain(data)
    num, out_data = connected_component(data)
    if len(num) != 0:
        volume = []
        for i in range(len(num)):
            volume.append(np_count(out_data[np.nonzero(out_data)], num[i]))
        counts += 1
    else:
        without_plaque_list.append(plaque.split('/')[6]+'/'+plaque.split('/')[7]+'/'+plaque.split('/')[8])
        continue
    print(num, volume)
    after_num, after_volume, after_data = remove_mix_region(num, volume, out_data)
    all_plaque_volumes.extend(after_volume)
    #print(all_plaque_volumes)
    #print(after_num, after_volume)
    #print(after_data.shape)
    #save_nii(plaque, after_data)
    dcm_folder = plaque.split('_CTA')[0]
    series = SERIES(series_path=dcm_folder, strict_check_series=True)
    img_tensor_int16 = series.get_image_tensor_int16()
    #print(img_tensor_int16)
    brightness = get_brightness(after_num, after_data, img_tensor_int16)
    print(num, brightness)
    all_plaque_brightness.extend(brightness)
    #print(all_plaque_brightness)
plaque_volumes_mean = np.mean(all_plaque_volumes)
plaque_brightness_mean = np.mean(all_plaque_brightness)
plaque_volumes_std = np.std(all_plaque_volumes, ddof=1)
plaque_brightness_std = np.std(all_plaque_brightness, ddof=1)
print(plaque_volumes_mean, plaque_volumes_std, plaque_brightness_mean, plaque_brightness_std)
#sns.distplot(all_plaque_volumes)
#sns.distplot(all_plaque_brightness)

#print(plaques)
#plaques_dir = os.path.join(plaques.split('mask_source/')[0], 'mask_source/mask_plaque.nii.gz')
#data = sitk.ReadImage('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1036623/FCE495EE/784DB1F2_CTA/mask_source/mask_plaque_dealt.nii.gz')
#data = sitk.GetArrayFromImage(data)
#print(data.shape)
#img = itk.imread('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1022837/5A047805/62D38185_CTA/mask_source/mask_plaque.nii.gz')#/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1022837/5A047805/62D38185_CTA/mask_source
#view(img)

def show_data(path):
    print(path)
    img = sitk.ReadImage(path)
    data = sitk.GetArrayFromImage(img)
    print(data.shape)
    data = np.unique(data)
    print(data)
#plaque_name = '/1036603/88130C9F/D6F78C10_CTA' #'/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/'+plaque_name+'/mask_source/mask_vessel_dealt.nii.gz'
vessel_nii = nib.load('/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1073332/B5664E68/5F654861_CTA/mask_source/mask_vessel.nii.gz')
vessel_plaque_nii = nib.load('/mnt/users/ffr_plaque_mask/1073332/5F654861/mask_vessel_plaque.nii.gz')
plque_nii = nib.load('/mnt/users/ffr_plaque_mask/1073332/5F654861/mask_plaque.nii.gz')
#show_data('/mnt/users/ffr_plaque_mask/1036603/D6F78C10/mask_vessel_plaque.nii.gz')
vessel = vessel_nii.get_fdata()
vessel_plaque = vessel_plaque_nii.get_fdata()
plaque = plque_nii.get_fdata()
pts1 = np.stack(np.where(vessel==1), axis=1)
pts2 = np.stack(np.where(vessel_plaque==1), axis=1)
pts3 = np.stack(np.where(vessel_plaque==3), axis=1)
view(point_sets=[pts1, pts2, pts3])

/mnt/DrwiseDataNFS/drwise_runtime_env/data1/inputdata/1036623/FCE495EE/784DB1F2_CTA/mask_source/mask_plaque.nii.gz
[5, 3, 1, 13, 7, 12, 2, 11, 6, 8, 4, 10, 9] [82, 74, 68, 47, 46, 40, 24, 23, 16, 6, 3, 1, 1]
[5, 3, 1, 13, 7, 12, 2, 11, 6, 8, 4, 10, 9] [959.0, 775.0, 350.0, 492.0, 831.0, 731.0, 360.0, 756.0, 656.0, 703.0, 971.0, 408.0, 566.0]
33.15384615384615 28.675907872423018 658.3076923076923 210.42353504911017


Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …